In [5]:
import pandas as pd

In [6]:
sp500 = pd.read_excel('data/s&p500.xlsx')
sp500_top10_holders = pd.read_excel('data/sp500_top10_holders.xlsx')

In [ ]:
# TODO fix headers and indexes

In [ ]:
# Group sp500_top10_holders by 'symbol' and sum the ['shares', '%', 'value'] columns
sp500_top10_holders_sum = sp500_top10_holders.groupby(1)[[4, 6, 7]].sum()
sp500_top10_holders_sum.head()

# Treemap

# Circular Packing